In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# 1. Load CSV data
def load_data(csv_file, target_column, batch_size):
    # Read CSV file
    df = pd.read_csv(csv_file,sep=';')
    
    # Split data into features and target
    X = df.drop(columns=[target_column,'id']).values
    y = df[target_column].values

    # Normalize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Convert to tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2)

    # Create DataLoader for batching
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader


In [3]:
# 2. Dynamic Neural Network
class DynamicNeuralNet(nn.Module):
    def __init__(self, input_size, layer_sizes, output_size):
        super(DynamicNeuralNet, self).__init__()
        layers = []
        previous_size = input_size

        # Dynamically create hidden layers
        for size in layer_sizes:
            layers.append(nn.Linear(previous_size, size))
            layers.append(nn.ReLU())
            # layers.append(nn.Dropout(0.3))
            previous_size = size

        # Output layer
        layers.append(nn.Linear(previous_size, output_size))

        # Combine layers
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [4]:
# 3. Train function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)
        
        # Validation loop
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)
        
        # Print stats for the epoch
        train_accuracy = correct_train / total_train
        val_accuracy = correct_val / total_val
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/total_train:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}')

In [6]:
csv_file = 'C:/Users/ASUS/NextgenAI/Projects/Cardio_data/cardio_train.csv'  # Path to your CSV file
target_column = 'cardio'  # Column name of the target variable in the CSV file
input_size = 11  # Number of input features
layer_sizes = [256, 256, 128]  # Layers: [256 neurons, 128 neurons, 64 neurons]
output_size = 2 # Number of output classes (e.g., for classification)
learning_rate = 0.002
batch_size = 32
num_epochs = 20

# Load data
train_loader, val_loader = load_data(csv_file, target_column, batch_size)
    
# Initialize model, loss function, and optimizer
model = DynamicNeuralNet(input_size=input_size, layer_sizes=layer_sizes, output_size=output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch [1/20], Loss: 0.0181, Train Accuracy: 0.7108, Val Accuracy: 0.7139
Epoch [2/20], Loss: 0.0172, Train Accuracy: 0.7314, Val Accuracy: 0.7269
Epoch [3/20], Loss: 0.0171, Train Accuracy: 0.7325, Val Accuracy: 0.7288
Epoch [4/20], Loss: 0.0170, Train Accuracy: 0.7334, Val Accuracy: 0.7304
Epoch [5/20], Loss: 0.0170, Train Accuracy: 0.7338, Val Accuracy: 0.7319
Epoch [6/20], Loss: 0.0170, Train Accuracy: 0.7352, Val Accuracy: 0.7324
Epoch [7/20], Loss: 0.0169, Train Accuracy: 0.7368, Val Accuracy: 0.7270
Epoch [8/20], Loss: 0.0169, Train Accuracy: 0.7364, Val Accuracy: 0.7303
Epoch [9/20], Loss: 0.0169, Train Accuracy: 0.7366, Val Accuracy: 0.7304
Epoch [10/20], Loss: 0.0168, Train Accuracy: 0.7372, Val Accuracy: 0.7253
Epoch [11/20], Loss: 0.0168, Train Accuracy: 0.7375, Val Accuracy: 0.7311
Epoch [12/20], Loss: 0.0168, Train Accuracy: 0.7385, Val Accuracy: 0.7317
Epoch [13/20], Loss: 0.0168, Train Accuracy: 0.7376, Val Accuracy: 0.7314
Epoch [14/20], Loss: 0.0168, Train Accuracy: 0.

KeyboardInterrupt: 